In [ ]:
import requests
import urllib.request
import time
import bs4
from bs4 import BeautifulSoup, NavigableString, Tag
import json
import pandas as pd
import numpy as np
from pprint import pprint

import json
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import re 
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [ ]:
url = 'https://www.kernel.org/doc/man-pages/'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
url_man_1_8 =['https://man7.org/linux/man-pages/dir_section_1.html', 'https://man7.org/linux/man-pages/dir_section_8.html']

In [ ]:
def extract_pars(pre):
    pars_text = pre.text.split('\n')
    pars_indices = []
    for p in range(len(pars_text)):
        if len(pars_text[p])-len(pars_text[p].lstrip())==7 and (pars_text[p].split(' ')[7].startswith('−') or 
                                                            pars_text[p].split(' ')[7].startswith('-')):
            pars_indices.append(p)
        elif len(pars_text[p])-len(pars_text[p].lstrip())==6 and (pars_text[p].split(' ')[6].startswith('−') or 
                                                            pars_text[p].split(' ')[6].startswith('-')):
            pars_indices.append(p)
        elif len(pars_text[p])-len(pars_text[p].lstrip())==5 and (pars_text[p].split(' ')[5].startswith('−') or 
                                                            pars_text[p].split(' ')[5].startswith('-')):
            pars_indices.append(p)
    extra_pars = ['-help', '--help','-help','−−help' '-version','-?','--?', '--version','−version', '−−version', '-V, --version',
                 '-?, --help', '−?, −−help','-V,--version', '-?,--help','−?,−−help', '-H,--HELP', '--HELP',  '−−VERSION',
                   '−VERSION', '−−VERSION','-?,--HELP', '−?,−−HELP' , '-h,--HELP']
    #extra_pars = []
    pars = dict()
    if len(pars_indices)>=1:
        for p in range(1,len(pars_indices)):
            start_ind = pars_indices[p-1]
            end_ind = pars_indices[p]
            parm = pars_text[start_ind].split()

            if parm[0] not in extra_pars and len(parm)>3:
                parm_key = parm[0]
                parm_desc = parm[1:]
                parm_desc += pars_text[start_ind+1:end_ind]
                parm_desc = ' '.join(parm_desc)
                pars[parm_key]=parm_desc
            elif parm[0] not in extra_pars:
                parm_key = ' '.join(parm)
                parm_desc = pars_text[start_ind+1:end_ind]
                parm_desc = ''.join(parm_desc)
                pars[parm_key]=parm_desc
        last_ind = pars_indices[-1]
        tot = len(pars_text)
        last_ind_key = pars_text[last_ind].split()
        if last_ind_key[0] not in extra_pars and len(last_ind_key)>4:
            parm_key = last_ind_key[0]
            parm_desc = last_ind_key[1:]
            parm_desc = ' '.join(parm_desc)
            for r in range(1,tot-last_ind):
                if len(pars_text[last_ind+r])>0:
                    parm_desc+=pars_text[last_ind+r]
                else:
                    break
            pars[parm_key] = parm_desc
        elif last_ind_key[0] not in extra_pars:
            parm_key = last_ind_key[0]
            parm_desc = ''  
            for r in range(1,tot-last_ind):
                if len(pars_text[last_ind+r])>0:
                    parm_desc+=pars_text[last_ind+r]
                else:
                    break
            pars[parm_key] = parm_desc
        else:
            pass
        
    else:
        pars['None']='None'
    
    return pars
    

In [ ]:
def extract_synopsis(pre):
    syn = pre.text.split('\n')  
    syn_pars = []
    for si in range(len(syn)):
        if len(syn[si])>1:
            syn_pars.append(syn[si].strip())
    return syn_pars


In [ ]:
data ={}
c=0
data['commands'] = []
for new_url in url_man_1_8:
    #new_url =l.a.get('href')
    response = requests.get(new_url)
    soup = BeautifulSoup(response.content.decode('utf-8','ignore'), 'html.parser')
    base_url = 'https://man7.org/linux/man-pages/'
    for h1 in soup.find_all('a'):
        if h1.get('href').startswith('./man'):
            new_urls = base_url+h1.get('href')[2:]
            response = requests.get(new_urls)
            soup = BeautifulSoup(response.content.decode('utf-8','ignore'), 'html.parser')
            comd_name = soup.title.text.split('(')[0]
            comd_name_one = soup.title.text.split('(')[1]

            parameters = dict()
            headers_names = []
            for h2 in soup.findAll('h2'):
                headers_names.append(h2.a.get('href'))

            
            if ('#SYNOPSIS' in headers_names or '#Synopsis' in headers_names) and (comd_name_one.startswith('1)') or comd_name_one.startswith('8)')):
                print(comd_name)
                for h2 in soup.findAll('h2'):
                    if h2.a.get('href')=='#NAME' or h2.a.get('href')=='#Name':
                        if h2.next_sibling.name=='pre':
                            tn = h2.next_sibling
                            comd_desc=tn.text
                        else:
                            pass
                    elif h2.a.get('href')=='#DESCRIPTION' or h2.a.get('href')=='#Description':
                        if h2.next_sibling.name=='pre':
                            tt = h2.next_sibling
                            comd_desc+=tt.get_text()
                        else:
                            pass
                    elif h2.a.get('href')=='#OPTIONS' or h2.a.get('href')=='#Options':
                        #parameters = extract_parameters_desc(h2)
                        if h2.next_sibling.name=='pre':
                            pre_p = h2.next_sibling
                            pp = pre_p.text.split('\n')
                            len_pp = len(pp)
                            if len_pp<=3 and ( 'None' in pp[1] or len(pp[1]) ==0):
                                parameters['None']='None'
                       
                            else: 
                                parameters = extract_pars(pre_p)
                                
                        else:
                            pass
                
                    elif (h2.a.get('href')=='#SYNOPSIS' or h2.a.get('href')=='#Synopsis') and h2.next_sibling.name=='pre':
                            pre_s = h2.next_sibling
                            synopsis_data = extract_synopsis(pre_s)
        
                
                if len(parameters)<1:
                    c+=1
                    parameters = extract_pars(tt)
                    comd_desc = ''
                    comd_desc+=tt.get_text()
                    comd_desc = re.split(r'\n\n\s+-',comd_desc)[0]
                            
                comd_desc = comd_desc.replace('\n', '')
                comd_desc = re.sub(' +', ' ', comd_desc)
                data['commands'].append({
                 'commands': comd_name,
                'description':comd_desc,
                 'synopsis':synopsis_data,
                'parameters':parameters})
                 

In [ ]:
with open('linux_data_raw.txt', 'w') as outfile:
    json.dump(data, outfile)